# Write Experiments

This file is used to generate JSON files for running experients

JSON files will be written for main.py to read arguments from, rather than going from the command-line

In [5]:
import pickle
import pandas as pd
from datetime import datetime
import os

In [12]:
exp_name = 'nav_0_obj'
config = {
    'wandb_project_name': 'nav1',
    'exp_name': exp_name,
    'save_name': 'nav_0_obj',
    'num_env_steps': 10000,
    'env_name': 'NavEnv-v0',
    'recurrent_policy': True,
    'algo': 'ppo',
    'num_mini_batch': 4,
    'num_processes': 4,
    'env_kwargs': {'num_objects': 0}
}

pickle.dump(config, open('experiment_configs/' + exp_name, 'wb'))

In [9]:
files = os.listdir('experiment_configs/')

In [10]:
for file in files:
    if file not in ['.ipynb_checkpoints', 'archive']:
        print(file)

hello_world


## Scheduler functions

These functions are found in scheduler.py (testing them here), used to run and track the experiments in experiment_configs

In [2]:
from scheduler import *

In [3]:
run_experiment('hello_world')